# TimerMixerPP Tutorial

##  Environment Setup

In [ ]:

pip install -r requirements.txt

### eval -> eval_mixerpp.py

#### 1 Package Import

In [ ]:
import json
import os
import argparse
import numpy as np
import logging
import torch
import torch.distributed as dist
from torch.utils.data import DistributedSampler, DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM
from Modeling.datasets.benchmark_dataset import ChannelEvalDataset

#### 2 eval_mixerpp.py

`UnifiedTS` is a utility class for loading and performing inference with time series forecasting models. By initializing this class, you can easily load a model onto a specified device and perform batch predictions, making it suitable for model evaluation or real-world inference scenarios.

In [ ]:
class UnifiedTS:
    def __init__(self, model_path, device, context_length, prediction_length, **kwargs):
        from Modeling.models.modeling_UnifiedTS import UnifiedTSForPrediction,UnifiedTSConfig
        self.model = UnifiedTSForPrediction.from_pretrained(
            model_path,  
            # device_map="auto",
            torch_dtype="auto"
        )
        self.device = device
        self.model = self.model.to(self.device) 
        self.prediction_length = prediction_length
        self.model.eval()

    def predict(self, batch):
       
        inputs = batch['inputs'].to(self.device)
        labels = batch['labels'].to(self.device)
        outputs = self.model(input_ids=inputs, labels=labels)

        preds = outputs.logits if isinstance(outputs.logits, list) else [outputs.logits]
        labels_slices = [labels[:, :pl] for pl in self.model.config.pred_len]
        return preds, labels_slices

#### Model Evaluation Workflow
The `evaluate` function provides a complete workflow for evaluating a time series forecasting model on a benchmark dataset. The process includes model loading, data preparation, batch inference, and metric calculation.

##### 1. Initialize Model and Device
First, set up the device (GPU or CPU) and load the forecasting model using the `UnifiedTS` class.

In [ ]:
def evaluate(args):
    batch_size = args.batch_size
    context_length = args.context_length
    prediction_lengths = [96, 192, 336, 720]  
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"
    model = UnifiedTS(
        args.model_path,
        device,
        context_length=context_length,
        prediction_length=prediction_lengths,  
        channel_mixing=args.channel_mixing
    )

##### 2. Prepare Evaluation Metrics
Create a dictionary to store MSE and MAE metrics for each prediction length.

In [ ]:
global_metrics = {pl: {'mse': 0.0, 'mae': 0.0, 'count': 0} for pl in prediction_lengths}


##### 3. Loop Over Prediction Lengths and Evaluate
For each prediction length, load the evaluation dataset, create a DataLoader, and compute metrics batch by batch.

In [ ]:
for pl in prediction_lengths:
        dataset = ChannelEvalDataset(
            args.data,
            context_length=context_length,
            prediction_length=pl
        )
        test_dl = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2,
            prefetch_factor=2,
            drop_last=False
        )
        local_mse = 0.0
        local_mae = 0.0
        local_count = 0
        with torch.no_grad():
            for batch in tqdm(test_dl, desc=f"Testing pl={pl}"):
                all_preds, all_labels = model.predict(batch)
                for pred, label, pred_len in zip(all_preds, all_labels, model.model.config.pred_len):
                    if pred_len == pl:
                        pred = pred.squeeze(-1)
                        label = label.squeeze(-1)
                        local_mse += torch.sum((pred - label) ** 2).item()
                        local_mae += torch.sum(torch.abs(pred - label)).item()
                        local_count += pred.numel()
                        break
        if local_count > 0:
            global_metrics[pl]['mse'] = local_mse / local_count
            global_metrics[pl]['mae'] = local_mae / local_count
            global_metrics[pl]['count'] = local_count

##### 4. Print and Log the Evaluation Results
After evaluation, print and log the final results for each prediction length.

In [ ]:
    result = {
        'model': args.model_path,
        'data': args.data,
        'context_length': context_length,
        'metrics': {}
    }

    for pl in prediction_lengths:
        if global_metrics[pl]['count'] > 0:
            result['metrics'].update({
                f'mse_{pl}': global_metrics[pl]['mse'],
                f'mae_{pl}': global_metrics[pl]['mae']
            })

    logging.info(json.dumps(result, indent=2))
    print("\nFinal Results:")
    for pl in prediction_lengths:
        print(f"PredLen {pl}: MSE={global_metrics[pl]['mse']:.4f}, MAE={global_metrics[pl]['mae']:.4f}")


#### Running the Evaluation Script
You can evaluate your trained model using the `eval_mixerPP.py` script. Specify the dataset path, enable channel mixing, set the batch size, and provide the model path as follows:

In [ ]:
python eval_mixerPP.py \
    -d data/test/data_etth1_train/ \
    --channel_mixing True \
    --batch_size 1024 \
    --model_path logs/timemixerpp

#### Fine-tuning Process
This project supports fine-tuning a pretrained time series forecasting model on these datasets . The fine-tuning workflow is managed by the `main.py` script, which configures the training process and launches model training via the `Runner` class.

##### 1. Fine-tuning Workflow Overview
- Argument Parsing:
`main.py` uses `argparse` to parse a wide range of training and model parameters, such as model path, data paths, batch size, learning rates, and more.
- Runner Initialization:
The script initializes a `Runner` object, which manages the training process.
- Training Start:
The `train_model` method is called to start fine-tuning, using the provided configuration.

In [ ]:
# main.py (simplified)
from Modeling.runner import Runner

if __name__ == '__main__':
    # Parse command-line arguments
    parser = argparse.ArgumentParser()
    # ... (argument definitions) ...
    args = parser.parse_args()

    runner = Runner(
        model_path=args.model_path,
        output_path=args.output_path,
        seed=args.seed,
    )

    runner.train_model(
        model_name=args.model_name,
        from_scratch=args.from_scratch,
        context_length=args.context_length,
        prediction_length=args.prediction_length,
        # ... (other training parameters) ...
    )


##### 2. Distributed Training Launcher
For efficient training on multiple GPUs, use the `torch_dist_run.py` script. This script automatically launches distributed training using `torchrun` if GPUs are available.

##### 3. Example Fine-tuning Command
You can start fine-tuning your model with a command like the following:

In [ ]:
python torch_dist_run.py main.py \
    --micro_batch_size 1 \
    --global_batch_size 8 \
    --channel_mixing True \
    -o logs/timemixerpp_traffic_finetune \
    -d data/train/data_electricity_train/ \
    -m /opt/tiger/UnifiedTSLib/logs/timemixerpp_1 \
    --val_data_path data/val/data_electricity_validation/


- `main.py`: The main training script.
- `--micro_batch_size`: Per-GPU batch size.
- `--global_batch_size`: Total batch size across all GPUs.
- `--channel_mixing True`: Enable channel mixing.
- `-o`: Output directory for logs and checkpoints.
- `-d`: Path to the training dataset.
- `-m`: Path to the pretrained model to be fine-tuned.
- `--val_data_path`: Path to the validation dataset.